# 1.加载向量数据库

In [6]:
import sys
# using zhipuai's embedding API
from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma

In [8]:
from dotenv import load_dotenv,find_dotenv
import os
_ = load_dotenv(find_dotenv())
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']
zhipuai_api_key

'32b0876eda94b85d1d0081eafb5aad2e.1tUeCGG9ElLcJiAO'

加载向量数据库，其中包含多个文档的Embedding

In [9]:
# define Embeddings
embedding = ZhipuAIEmbeddings()
# 向量数据库持久化路径
persist_directory = '../LLM/data_base/vector_db/chroma'
# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory, # 允许我们将persist_directory目录保存到本地磁盘
    embedding_function=embedding
    # 提供embedding_function用于新数据的处理和查询
)
print(f"向量数据库中存储的数量：{vectordb._collection.count()}")

向量数据库中存储的数量：40


In [11]:
question = "what means prompt engineering"
docs = vectordb.similarity_search(question,k = 4)

for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容：\n{doc.page_content}",end="\n----------\n")

检索到的第0个内容：
当您使用指令微调 LLM 时，您可以类比为向另一个人提供指令（假设他很聪明但不知道您任务的具体细节）。因此，当 LLM 无法正常工作时，有时是因为指令不够清晰。例如，如果您想问“请为我写一些关于阿兰·图灵( Alan Turing )的东西”，在此基础上清楚表明您希望文本专注于他的科学工作、个人生活、历史角色或其他方面可能会更有帮助。另外您还可以指定回答的语调, 来更加满足您的需求，可选项包括专业记者写作，或者向朋友写的随笔等。

如果你将 LLM 视为一名新毕业的大学生，要求他完成这个任务，你甚至可以提前指定他们应该阅读哪些文本片段来写关于 Alan Turing 的文本，这样能够帮助这位新毕业的大学生更好地完成这项任务。下一章你会看到提示词创建的两个原则，一是清晰明确，二是给LLM时间去思考。
----------
检索到的第1个内容：
当您使用指令微调 LLM 时，您可以类比为向另一个人提供指令（假设他很聪明但不知道您任务的具体细节）。因此，当 LLM 无法正常工作时，有时是因为指令不够清晰。例如，如果您想问“请为我写一些关于阿兰·图灵( Alan Turing )的东西”，在此基础上清楚表明您希望文本专注于他的科学工作、个人生活、历史角色或其他方面可能会更有帮助。另外您还可以指定回答的语调, 来更加满足您的需求，可选项包括专业记者写作，或者向朋友写的随笔等。

如果你将 LLM 视为一名新毕业的大学生，要求他完成这个任务，你甚至可以提前指定他们应该阅读哪些文本片段来写关于 Alan Turing 的文本，这样能够帮助这位新毕业的大学生更好地完成这项任务。下一章你会看到提示词创建的两个原则，一是清晰明确，二是给LLM时间去思考。
----------
检索到的第2个内容：
互联网上有很多有关提示词（Prompt, 本教程中将保留该术语）的材料，例如《30 prompts everyone has to know》之类的文章。这些文章主要集中在 ChatGPT 的 Web 界面上，许多人在使用它执行特定的、通常是一次性的任务。但是，我认为对于开发人员，LLM 的更强大功能是能通过 API 调用，从而快速构建软件应用程序。我认为这方面还没有得到充分的重视。实际上，我们在 DeepLearning.AI 的姊妹公司 AI Fund 的团队一直在

# 2.Create a LLM

In [13]:
import os
OPENAI_AIP_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = os.environ['BASE_URL']

In [15]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0,
                 openai_api_key = OPENAI_AIP_KEY,base_url = BASE_URL)
llm.invoke("please introduce yourself")

AIMessage(content='Hello, I am an AI digital assistant designed to help and assist with various tasks and inquiries. I can provide information, answer questions, and help with a wide range of topics. I am always here to assist you to the best of my abilities. Please feel free to ask me anything you need help with.', response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 10, 'total_tokens': 72}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ec17f54-bd57-40a2-98e4-c64934b5fd0b-0', usage_metadata={'input_tokens': 10, 'output_tokens': 62, 'total_tokens': 72})

# 3.构建检索问答链

In [17]:
from langchain.prompts import PromptTemplate
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，
不要试图编造答案。最多使用三句话。
答案尽量简明扼要。总是要在回答的最后说“谢谢你的提问”。
{context}
问题：{question}
"""
# 创建一个提示模板实例
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                template=template)

In [19]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(# 指定使用的语言模型和检索器，并传递提示模板
    llm,
    retriever=vectordb.as_retriever(), # 将vectordb转换为一个检索器，用于从数据库中检索相关的文档
                                       
    return_source_documents=True,     # 是否返回原文档
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT} # 指定了要使用的提示模板
)

# 4.检索问答链效果测试

In [20]:
question_1 = "hi"
question_2 = "智谱AI是一家什么公司"

In [21]:
result = qa_chain({"query":question_1})
print("大模型+知识库后回答question_1的结果：\n",result["result"])

大模型+知识库后回答question_1的结果：
 抱歉，我不知道答案。谢谢你的提问。


In [18]:
# LLM自己回答的效果
prompt_template = """please answer the question as followed:{}""".format(question_1)
llm.predict(prompt_template)

D:\Software\anaconda3\envs\AFT_new\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'南瓜书是一本关于计算机科学和编程的教材。它的名称来源于计算机科学领域的术语“南瓜书（Pumpkin Book）”，这个术语指代的是一本经典的教材《编译原理》（Compilers: Principles, Techniques, and Tools），因其封面描绘了一只戴着南瓜帽子的小猫而得名。因此，一般而言，当人们提到“南瓜书”时，他们指的是这本著名的编译原理教材。'

In [20]:
result = qa_chain({"query":question_2})
print("大模型+知识库后回答question_1的结果：\n",result["result"])

大模型+知识库后回答question_1的结果：
 我不知道智谱AI是一家什么公司。谢谢你的提问。


In [21]:
# LLM自己回答的效果
prompt_template = """please answer the question as followed:{}""".format(question_2)
llm.predict(prompt_template)

'智谱AI是一家人工智能公司。'

# 添加历史对话的记忆功能

## 1.Memory

In [22]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True   # 消息列表的形式返回聊天记录，而非单个字符
)

## 2.对话检索链

对话检索链在检索QA链的基础上，增加了处理对话历史的能力

In [28]:
from langchain.chains import ConversationalRetrievalChain
retriever = vectordb.as_retriever() #向量数据库的检索器
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever = retriever,
    memory = memory
)
question = "can I learn something about fine-tune"
result = qa({"question":question})
print(result['answer'])

Based on the given context, you can learn the following specific details about fine-tuning:

- The course will focus on the best practices for fine-tuning language models (LLM) and recommend using them in most scenarios.
- The course materials have contributions from both OpenAI and DeepLearning.ai teams.
- Several individuals from OpenAI, including Andrew Main, Joe Palermo, Boris Power, Ted Sanders, and Lillian Weng, were involved in brainstorming and reviewing the course materials.
- Geoff Ladwig, Eddy Shyu, and Tommy Nelson from DeepLearning.ai have also worked on the course.

Unfortunately, the given context does not provide further details about fine-tuning itself.


In [29]:
question = "why we need the knowledge in this aspect"
result = qa({"question":question})
print(result['answer'])

It is important to have knowledge about fine-tuning in the context of this course because the course focuses on best practices for fine-tuning language models. It is recommended to use these practices in most scenarios. Fine-tuning allows us to adapt pre-trained models to specific tasks or domains, improving their performance and making them more useful in practical applications. So, understanding fine-tuning techniques is crucial for effectively utilizing language models in this course.
